### Mount gdrive

### This cell is for Google colaboratory users.

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%cd /gdrive/My\ Drive/DNNTopology

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
/gdrive/My Drive/DNNTopology


### Parameter setting

In [0]:
class_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
firstLayerSize = 512
secondLayerSize = 512
outputNeuron = 10

outputDir = "./models/"
epochs = 30
batch_size = 32
model_name = outputDir + "firstLayerSize"+str(firstLayerSize)+"secondLayerSize"+str(secondLayerSize)+"outputNeuron"+str(outputNeuron)+ "class" + str(class_list) + ".h5"

### Import libraries

In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
%matplotlib inline

### Load CIFAR10 data set

In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Extract data in class_list
train_filter = np.where( [ (element in class_list) for element in y_train] )
test_filter = np.where( [ (element in class_list) for element in y_test] )
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

In [0]:
# Normalize image data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# Convert vectors to binary class
y_train = keras.utils.to_categorical(y_train, outputNeuron)
y_test = keras.utils.to_categorical(y_test, outputNeuron)

### Make model

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(firstLayerSize, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(secondLayerSize, activation = 'relu'))
model.add(Dense(outputNeuron, activation = 'softmax'))

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 64)       

### Train model

In [0]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
fit = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 11s 215us/step - loss: 1.8454 - acc: 0.3207 - val_loss: 1.5592 - val_acc: 0.4285
Epoch 2/30
50000/50000 [==============================] - 10s 207us/step - loss: 1.4731 - acc: 0.4672 - val_loss: 1.3164 - val_acc: 0.5285
Epoch 3/30
50000/50000 [==============================] - 11s 212us/step - loss: 1.3278 - acc: 0.5265 - val_loss: 1.3014 - val_acc: 0.5426
Epoch 4/30
50000/50000 [==============================] - 10s 209us/step - loss: 1.2198 - acc: 0.5669 - val_loss: 1.1160 - val_acc: 0.6038
Epoch 5/30
50000/50000 [==============================] - 10s 204us/step - loss: 1.1376 - acc: 0.5989 - val_loss: 1.0312 - val_acc: 0.6358
Epoch 6/30
50000/50000 [==============================] - 11s 218us/step - loss: 1.0703 - acc: 0.6237 - val_loss: 0.9920 - val_acc: 0.6535
Epoch 7/30
50000/50000 [==============================] - 10s 209us/step - loss: 1.0176 - acc: 0.6431 - val_loss: 0.

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

### Save model

In [0]:
model.save(model_name, include_optimizer = False)